# Conociendo los RDDs


En este notebook trabajaremos con los RDDs, que forman parte del Spark Core.La implementación de Spark Core es un **RDD (Resilient Distributed Dataset)** que es una colección de datos distribuidos en diferentes nodos del clúster que se procesan en paralelo.

Utilizaremos la API de PySpark, pero los conceptos aplican por igual a todas las APIs (Scala, R, etc)

### Inicialización de Spark en Notebooks

In [1]:
# Install spark-related dependencies
!wget -q  https://apache.osuosl.org/spark/spark-3.5.0/spark-3.5.0-bin-hadoop3.tgz
!tar xf spark-3.5.0-bin-hadoop3.tgz

!pip install -q findspark
!pip install pyspark
# Set up required environment variables

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.0-bin-hadoop3"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 1.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=0ce4c93d5ff82e380062b87802845ba4402bf988ac1fd89117fc3b9eb8e06788
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [2]:
import findspark
findspark.init() #para inicializar

import pandas as pd
import pyspark

In [3]:
from pyspark.sql import SparkSession

### Crear el SparkSession y el SparkContext

In [4]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master("local[*]")\
        .appName('PySpark_training')\
        .getOrCreate() #devuelve una sesion existente sino existe la crea

In [5]:
spark = SparkSession.builder.getOrCreate() #genera lo mismo que antes solo que con los valores por default
sc = spark.sparkContext

### Crear un RDD de una colección

In [6]:
num = [1,2,3,4,5]

num_rdd = sc.parallelize(num) #función para paralelizar
num_rdd.collect() #con collect, recogemos toda la lista de números

[1, 2, 3, 4, 5]

# Transformaciones
* Como sabemos, las Transformaciones son de naturaleza perezosa y no se ejecutarán hasta que se ejecute una Acción sobre ellas.
* Intentemos comprender las distintas transformaciones disponibles.

Para mas informacion, puedes apoyarte del siguiente link:
* https://keepcoding.io/blog/transformaciones-y-acciones-en-spark/


### map
* Esto mapeará su entrada a alguna salida basada en la función especificada en la función

In [7]:
double_rdd = num_rdd.map(lambda x : x * 2)
double_rdd.collect()

[2, 4, 6, 8, 10]

### filtro
* Para filtrar los datos en función de una determinada condición. Intentemos encontrar los números pares de num_rdd.

In [8]:
even_rdd = num_rdd.filter(lambda x : x % 2 == 0) #filtro por los elementos pares del rdd
even_rdd.collect()

[2, 4]

### distinct
* Esto devolverá elementos distintos de un RDD.

In [9]:
rdd1 = sc.parallelize([10, 11, 11, 13, 11, 10, 12])
dist_rdd = rdd1.distinct()
dist_rdd.collect()

[10, 12, 11, 13]

### reduceByKey
* Esta función reduce los pares de valores clave en función de las claves y una función determinada dentro de reduceByKey

In [10]:
pairs = [ ("a", 8), ("b", 3), ("c", 3), ("a", 5), ("b", 1), ("c", 4)]
pair_rdd = sc.parallelize(pairs)

output = pair_rdd.reduceByKey(lambda x, y : x + y)

result = output.collect()
print(*result, sep='\n')

('b', 4)
('c', 7)
('a', 13)


### sortByKey
* Esta función realizará la clasificación en un par (clave, valor) RDD basado en las claves. De forma predeterminada, la clasificación se realizará en orden ascendente.

In [11]:
pairs = [ ("a", 5), ("d", 7), ("c", 2), ("b", 3)]
raw_rdd = sc.parallelize(pairs)

sortkey_rdd = raw_rdd.sortByKey() #ascending=False
result = sortkey_rdd.collect()
print(*result,sep='\n')

# Para clasificar en orden descendente, pase  “ascending=False”.

('a', 5)
('b', 3)
('c', 2)
('d', 7)


# Acciones

* Las acciones son operaciones en RDD que se ejecutan inmediatamente. Mientras que las transformaciones devuelven otro RDD, las acciones devuelven estructuras de datos nativas

### count
* Esto contará el número de elementos en el RDD dado.

In [ ]:
num = sc.parallelize([1,2,4,5,2])
num.count()

5

### first
* Esto devolverá el primer elemento del RDD dado.

In [ ]:
num.first()

1

### Collect
* Esto devolverá todos los elementos para el RDD dado.


In [ ]:
num.collect()

[1, 2, 4, 5, 2]

### Take
* Esto devolverá el número de elementos especificados.

In [ ]:
num.take(3)

[1, 2, 4]